In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        self.hidden_dropout = nn.Dropout(0.5)

        self.conv1 = nn.Conv2d(3, 96, 11, 4, 2)
        self.conv2 = nn.Conv2d(96, 256, 5, 1, 2)
        self.conv3 = nn.Conv2d(256, 384, 3, 1, 1)
        self.conv4 = nn.Conv2d(384, 384, 3, 1, 1)
        self.conv5 = nn.Conv2d(384, 256, 3, 1, 1)
        
        self.pool = nn.MaxPool2d(3, 2)

        self.fc1 = nn.Linear(256*6*6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
    
    def forward(self, x):
        c1 = self.pool(F.relu(self.conv1(x)))
        c2 = self.pool(F.relu(self.conv2(c1)))
        c3 = F.relu(self.conv3(c2)) # Convoluciones sin MaxPool
        c4 = F.relu(self.conv4(c3))
        c5 = self.pool(F.relu(self.conv5(c4)))

        a = c5.view(c5.size(0), -1)  # Flatten
        a = self.hidden_dropout(a)

        h1 = F.relu(self.fc1(a))
        h1 = self.hidden_dropout(h1)
        h2 = F.relu(self.fc2(h1))
        h2 = self.hidden_dropout(h2)

        z = F.relu(self.fc3(h2))

        return z

In [18]:
import torchvision
import torchvision.transforms as transforms

# Transformaciones (AlexNet necesita 224x224)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)


In [19]:
import torch.optim as optim 

model = AlexNet(10)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 0.01, 0.9, weight_decay=0.0005)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # El modelo se pone en modo de entrenamiento
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * 64 + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # El modelo se pone en modo de test (evaluacion)
    model.eval()

    # Se desactiva el calculo del gradiente porque es test, no train
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 10
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.302940  [  128/50000]
loss: 2.301464  [ 6528/50000]
loss: 2.299802  [12928/50000]
loss: 2.271374  [19328/50000]
Test Error: 
 Accuracy: 28.0%, Avg loss: 1.961524 

Epoch 2
-------------------------------
loss: 1.906857  [  128/50000]
loss: 1.852206  [ 6528/50000]
loss: 1.596208  [12928/50000]
loss: 1.465136  [19328/50000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 1.433209 

Epoch 3
-------------------------------
loss: 1.526864  [  128/50000]
loss: 1.515997  [ 6528/50000]
loss: 1.293888  [12928/50000]
loss: 1.351556  [19328/50000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.262563 

Epoch 4
-------------------------------
loss: 1.170097  [  128/50000]
loss: 1.159874  [ 6528/50000]
loss: 1.063910  [12928/50000]
loss: 0.926640  [19328/50000]
Test Error: 
 Accuracy: 68.4%, Avg loss: 0.913187 

Epoch 5
-------------------------------
loss: 0.927423  [  128/50000]
loss: 0.927898  [ 6528/50000]
loss: 1.059761  [12928/50000]
loss: 0.830194  [19328